In [ ]:
import tensorflow as tf
import os
from tensorflow.keras import layers, Sequential
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import PIL

2025-05-19 16:24:49.466446: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_img_data(dir, size):
    imgs = []
    names = []
    for nombre in os.listdir(dir):
        ruta = os.path.join(dir, nombre)
        img = load_img(ruta, target_size=size)
        img_array = img_to_array(img).astype(np.uint8)
        imgs.append(img_array)
        names.append(os.path.splitext(nombre)[0])  # sin extensión
    return imgs, names

In [3]:
def save_img_data(imgs, names, dst, it , original:bool):
    typ = "orig" if original else "aug"
    for i, img_array in enumerate(imgs):
        img = PIL.Image.fromarray(img_array)
        name = f"{names[i]}_{typ}.jpg" if it ==0 else f"{names[i]}_{it}_{typ}.jpg"
        img.save(os.path.join( dst, name ))

In [5]:
def data_augmentation(in_dir, out_dir, size, it, save_orig:bool = False):
    os.makedirs(out_dir, exist_ok=True)

    # Carga de imagenes de la carpeta
    imgs, names = load_img_data(in_dir, size)

    # Convertimos a batch tensor 
    batch = tf.convert_to_tensor(imgs, dtype=tf.uint8)

    # Parámetros para augmentación
    data_augmentation = Sequential([
        layers.CenterCrop(512, 512), # Recorta el centro, priorizando el elemento central
        layers.Resizing(256, 256),   # Se reduce recién la imagen
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
    ])

    # Convertimos a float32 para aplicar augmentación (requerido por Keras)
    batch_float = tf.cast(batch, tf.float32)

    # Paso 4: Aplicar augmentación vectorizada
    batch_aug = data_augmentation(batch_float, training=True)
    batch_aug = tf.clip_by_value(batch_aug, 0.0, 255.0)
    batch_aug = tf.cast(batch_aug, tf.uint8)
    batch_aug = batch_aug.numpy()

    # Guardamos los datos originales en el dataset
    if save_orig:
        save_img_data(imgs, names, out_dir, it, original=True)

    # Guardamos augmentacion
    save_img_data(batch_aug, names, out_dir, it, original=False)

    msg = _ 
    if save_orig: 
        msg = f"Guardadas {len(batch)} imágenes originales y {len(batch_aug)} augmentadas en '{out_dir}'"
    else: 
        msg = f"Guardadas {len(batch_aug)} imgágenes augmentated en {out_dir}"
    print(msg)

In [6]:
dataset = "castanas_ds"
dataset_aug = "castanas_aug_ds"
class1 = "comible"
class2 = "no_comible"
size = (256,256)

# Aumento de datos clase 1
data_augmentation(f"{dataset}/{class1}", f"{dataset_aug}/{class1}", it=0, size=size, save_orig=True )
data_augmentation(f"{dataset}/{class1}", f"{dataset_aug}/{class1}", it=1, size=size )
data_augmentation(f"{dataset}/{class1}", f"{dataset_aug}/{class1}", it=2, size=size )

# Auento de datos clase 2
data_augmentation(f"{dataset}/{class2}", f"{dataset_aug}/{class2}", it=0, size=size, save_orig=True )
data_augmentation(f"{dataset}/{class2}", f"{dataset_aug}/{class2}", it=1, size=size )
data_augmentation(f"{dataset}/{class2}", f"{dataset_aug}/{class2}", it=2, size=size )

2025-05-19 16:26:10.556107: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 581959680 exceeds 10% of free system memory.
2025-05-19 16:26:39.497571: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1163919360 exceeds 10% of free system memory.


Guardadas 370 imágenes originales y 370 augmentadas en 'castanas_aug_ds/comible'


2025-05-19 16:27:26.458488: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 581959680 exceeds 10% of free system memory.
2025-05-19 16:27:56.406508: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1163919360 exceeds 10% of free system memory.


Guardadas 370 imgágenes augmentated en castanas_aug_ds/comible


2025-05-19 16:28:41.591624: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 581959680 exceeds 10% of free system memory.


Guardadas 370 imgágenes augmentated en castanas_aug_ds/comible
Guardadas 177 imágenes originales y 177 augmentadas en 'castanas_aug_ds/no_comible'
Guardadas 177 imgágenes augmentated en castanas_aug_ds/no_comible
Guardadas 177 imgágenes augmentated en castanas_aug_ds/no_comible
